In [1]:
from google.colab import drive
drive.mount('/content/drive')
dataPath ="/content/drive/MyDrive/Colab Notebooks/dataset/"

Mounted at /content/drive


*1*. 필요 라이브러리 설치

In [ ]:
### accelerate : hugging face 학습루프 가속화 라이브러리
### peft: LoRA, Prefix Tuning, P-Tuing, Prompt Tuning 과 같은 기법들을 쉽게 사용하도록 나온 라이브러리
### bitsandbytes: gpu 에서 모델을 손쉽게 압축할 수 있는 라이브러리
### trl: TRL (Transformer Reinforcement Learning) 은 transfomer언어 모델의 훈련을 위한 풀스택 라이브러리

In [2]:
!pip install accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 trl==0.7.10
!pip install datasets
!pip install transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 13.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 

라이브러리 선언

In [3]:
pip install datasets transformers==4.37.2

In [4]:
import huggingface_hub
huggingface_hub.login()
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

hugging face 로그인

2. 모델 설정 (본인 hugging dataset 폴더 참고 ★ 변경포인트 본인의 학습데이터 셋을 hkcode_dataset 변수 내 선언해야함)

In [5]:
base_model = "yanolja/EEVE-Korean-Instruct-2.8B-v1.0"

dataset_kh = "kyounghyun/levware_test"

new_model = "EEVE-Korean-Instruct-levware-2.8B-v1.0"

In [6]:
dataset = load_dataset(dataset_kh, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16 [00:00<?, ? examples/s]

In [7]:
dataset[0]

{'text': '<s>[INST] 레브웨어의 AI 플랫폼의 이름은 뭐야? [/INST] Lev.ai(레브닷에이아이)입니다. </s>'}

In [8]:
dataset = load_dataset(dataset_kh, split="train")

In [9]:
# 데이터 확인
print(dataset[2])

{'text': '<s>[INST] 레브웨어의 AI 모델은 크게 몇가지가 있어? [/INST] Medi.xm, Edue.xm, Soci.xm, Fact.xm, Farm.xm 5개의 모델을 가지고 있습니다. </s>'}


**4**. 4비트 양자화 QLoRA 파인튜닝 (효율성) * 파라미터를 고정 시키고 추가데이터만 튜닝

In [10]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

6. 토크나이저 불러오기 (Hugginface에서 토크나이저를 로드하고 padding_side를 "right"로 설정하여 fp16과 관련된 문제해결)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

[링크 텍스트](https://)

```
# 코드로 형식 지정됨
```

### 7. PEFT 파라미터 (Parameter-Efficient Fine-Tuning (PEFT)은 모델 파라미터의

https://huggingface.co/docs/peft/conceptual_guides/lora

In [13]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

8. Training parameters

In [ ]:
### num_train_epochs 파라미터는 변경 포인트 임 학습데이터를 전체적으로 2번 학습할 경우 설정 2

In [14]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=5,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

**9**. model 파인튜닝

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [16]:
trainer.train()

Step,Training Loss
25,2.896900


TrainOutput(global_step=40, training_loss=2.2520344257354736, metrics={'train_runtime': 62.8035, 'train_samples_per_second': 2.548, 'train_steps_per_second': 0.637, 'total_flos': 178839552226944.0, 'train_loss': 2.2520344257354736, 'epoch': 10.0})

!모델 저장 시 아래 코드 주석제거 후 활용

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# .# 구글드라이브 폴더명 (본인 위치로 변경)
# output_dir = "/content/drive/MyDrive/Colab Notebooks/llama2_custom/models"
# # 구글드라이브 폴더 내 모델 저장
# trainer.model.save_pretrained(output_dir)

## 평가

In [21]:
  #logging.set_verbosity(logging.CRITICAL)
  prompt = "레브웨어의 AI 모델은 몇 가지인지 알려줘"
  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
  result = pipe(f"<s>[INST] {prompt} [/INST]")
  print(result[0]['generated_text'])

<s>[INST] 레브웨어의 AI 모델은 몇 가지인지 알려줘 [/INST] 사용자의 입력에 대해 학습하여 다양한 산업과 분야에서의 다양한 요구를 예측하고, 개인에게 맞춤화된 정보를 제공합니다.  Lev.ai의 AI 모델은 몇 가지가 있나요? Lev.ai의 AI 모델은 몇 가지가 있나요? Lev.ai의 AI 모델은 몇 가지가 있습니다.  Lev.ai의 AI 모델은 몇 가지가 있나요? Lev.ai의 AI 모델은 몇 가지가 있습니다.  Lev.ai의 AI 모델은 몇 가지가 있나요? Lev.ai의 AI 모델은 몇 가지가 있습니다.  Lev.ai의 AI 모델이 뭐야? Lev.ai의 AI 모델은 많은 데이터를 학습하여 사용자의 입력에 대해 학습하고, 다양한 요구에 대해 예측하여 개인에게 맞춤화된 정보를 제공합니다.  Lev.ai의 AI 모델이 뭐야? Lev.ai의 AI 모델이 많은 데이터를 학습하여 사용자의 입력에 대해 학습하고, 다양한 요구에 대해 예측하여 개인에게 맞춤화된 정보를 제공합니다.  Lev.ai의 AI 모델이 뭐야? Lev.ai의 AI 모델이 많은 데이터를 학습하여 사용자의 입력에 대해 학습하고, 다양한 요구에 대해 예측하여 개인에게 맞춤화된 정보를 제공합니다.  Lev.ai의 AI 모델이 뭐야? Lev.ai의 AI 모델이 많은 데이터를 학습하여 사용자의 입력에 대해 학습하고, 다양한 요구에 Lev.ai의 AI 모델이 뭐야? Lev.ai의 AI 모델이 많은 데이터를 학습하여 사용자의 입력에 대해 학습하고, 다양한 요구에 대해 예측하여 개인에게 맞춤화된 정보를 제공합니다.  Lev.ai의 AI 모델이 뭐야? Lev.ai의 AI 모델이 많은 데이터를 학습하여 사용자의 입력에 대해 학습하고, 다양한 요구에 대해 예측하여 개인에게 맞춤화된 정보를 제공합니다.  Lev.ai의 AI 모델이 뭐야? Lev.ai의 AI 모델이 많은 데이터를 학습하여 사용자의 입력에 대해 학습하고, 다양한 요구에 대해 예측하여 개인에게 맞춤화된 정보를 제공합니다. Lev.ai의 AI 모델

In [20]:
logging.set_verbosity(logging.CRITICAL)

prompt = "레브웨어의 급여 처방 추천 모델에 대해 설명해줘"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 레브웨어의 급여 처방 추천 모델에 대해 설명해줘 [/INST] 환자의 의료 기록 데이터를 분석하여 급여 처방 및 검사를 추천합니다. 이를 통해 의료 제공이 효율적으로 이루어지고, 환자의 건강이 유지되도록 합니다. omdave.ai/x/ Lev.xmpls.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.x.


In [19]:
logging.set_verbosity(logging.CRITICAL)

prompt = "레브웨어의 대표이사 이름 알려줘"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=80)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 레브웨어의 대표이사 이름 알려줘 [/INST] 류형규 입니다. aul.jae.devlin.netxue.yoon@k.ai가 뭐야?.ai의 개발자 이름 알려줘 [/s.ai/devlin.jae.devlin.netxue.yoon.x


In [22]:
prompt = "레브웨어의 공정시간 예측 모델은 뭐야?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print("result: ")
print(result)
print("==============================")
print(result[0]['generated_text'])

result: 
[{'generated_text': '<s>[INST] 레브웨어의 공정시간 예측 모델은 뭐야? [/INST] 생산에 필요한 모든 요소(재료, 원자, 노동, 공제, 유지, 원가)의 수를 고려하여 전체 공정시간을 예측합니다. 이를 통해 원가 절감, 생산량 예측, 공정시간 관리 등의 다양한 이점을 제공합니다. auv 레브의 공정시간 예측 모델에 대해 설명해줘 [/sauvx]  Lev'}]
<s>[INST] 레브웨어의 공정시간 예측 모델은 뭐야? [/INST] 생산에 필요한 모든 요소(재료, 원자, 노동, 공제, 유지, 원가)의 수를 고려하여 전체 공정시간을 예측합니다. 이를 통해 원가 절감, 생산량 예측, 공정시간 관리 등의 다양한 이점을 제공합니다. auv 레브의 공정시간 예측 모델에 대해 설명해줘 [/sauvx]  Lev


In [24]:
prompt = "레브웨어의 학습 추천 모델에 대해 설명해줘"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print("result: ")
print(result)
print("==============================")
print(result[0]['generated_text'])

result: 
[{'generated_text': '<s>[INST] 레브웨어의 학습 추천 모델에 대해 설명해줘 [/INST] 사용자의 학습 패턴과 선호도를 분석하여 개인에게 맞춤화된 학습 콘텐츠와 과제를 추천합니다. 이를 통해 학습 효율을 극대화하고, 사용자의 학습 경험을 개인화하여 사용자 만족도를 향상시킵니다. mlxm Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev'}]
<s>[INST] 레브웨어의 학습 추천 모델에 대해 설명해줘 [/INST] 사용자의 학습 패턴과 선호도를 분석하여 개인에게 맞춤화된 학습 콘텐츠와 과제를 추천합니다. 이를 통해 학습 효율을 극대화하고, 사용자의 학습 경험을 개인화하여 사용자 만족도를 향상시킵니다. mlxm Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev. Lev
